In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

# 设置中文字体支持
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号

# 读取数据
file_path = r"./data/raw/training_data.xlsx"  # 请根据实际数据文件路径调整
msw_data = pd.read_excel(file_path, sheet_name='msw_result')

# 首先划分国家，避免数据泄露
np.random.seed(888)  # 设置随机种子确保结果可复现

# 按地区和收入组分组国家
country_groups = msw_data.groupby(['Country Name']).agg({
    'Region': 'first',
    'Income Group': 'first'
}).reset_index()

# 创建分层组合
country_groups['Strata'] = country_groups['Region'] + '_' + country_groups['Income Group']
strata_counts = country_groups['Strata'].value_counts()

# 计算每个分层需要抽取的国家数量（总体15%）
test_size = int(len(country_groups) * 0.15)
test_countries = []

# 按比例从每个分层中抽样
for strata, count in strata_counts.items():
    strata_countries = country_groups[country_groups['Strata'] == strata]['Country Name'].tolist()
    # 计算该分层需要抽取的国家数量（向上取整确保至少抽取一个）
    n_sample = max(1, int(np.ceil(count / len(country_groups) * test_size)))
    # 如果该分层国家数量不足，则全部选择
    if len(strata_countries) <= n_sample:
        sampled = strata_countries
    else:
        sampled = np.random.choice(strata_countries, size=n_sample, replace=False)
    test_countries.extend(sampled)

# 确保总数不超过预期的15%
if len(test_countries) > test_size:
    test_countries = np.random.choice(test_countries, size=test_size, replace=False)

print(f"测试集国家数量: {len(test_countries)}")
print(f"测试集国家列表: {test_countries}")

# 分析测试集的分层分布
test_strata = country_groups[country_groups['Country Name'].isin(test_countries)]
print("\n测试集分层分布:")
print(test_strata.groupby(['Region', 'Income Group']).size())

# 划分训练集和测试集原始数据
train_data_raw = msw_data[~msw_data['Country Name'].isin(test_countries)]
test_data_raw = msw_data[msw_data['Country Name'].isin(test_countries)]

print(f"\n训练集原始数据大小: {len(train_data_raw)}")
print(f"测试集原始数据大小: {len(test_data_raw)}")

In [ ]:
#分别对训练集和测试集进行特征工程
# 训练集特征工程
train_data = train_data_raw.dropna(subset=['MSW']).copy()
train_data['MSW_log'] = np.log1p(train_data['MSW'])

# 只使用训练集信息创建特征
train_year_min = train_data['Year'].min()
train_data['year_trend'] = train_data['Year'] - train_year_min
train_data['year_trend_squared'] = train_data['year_trend'] ** 2

# 创建地区-时间交互特征（仅使用训练集信息）
for region in train_data['Region'].unique():
    train_data[f'trend_region_{region}'] = (train_data['Region'] == region) * train_data['year_trend']

# 创建收入组-时间交互特征（仅使用训练集信息）
for income in train_data['Income Group'].unique():
    train_data[f'trend_income_{income}'] = (train_data['Income Group'] == income) * train_data['year_trend']

# 测试集特征工程 - 使用训练集的信息
test_data = test_data_raw.dropna(subset=['MSW']).copy()
test_data['MSW_log'] = np.log1p(test_data['MSW'])
test_data['year_trend'] = test_data['Year'] - train_year_min  # 使用训练集的最小年份
test_data['year_trend_squared'] = test_data['year_trend'] ** 2

# 创建测试集的地区-时间交互特征（使用训练集中的地区列表）
for region in train_data['Region'].unique():
    test_data[f'trend_region_{region}'] = (test_data['Region'] == region) * test_data['year_trend']

# 创建测试集的收入组-时间交互特征（使用训练集中的收入组列表）
for income in train_data['Income Group'].unique():
    test_data[f'trend_income_{income}'] = (test_data['Income Group'] == income) * test_data['year_trend']

print(f"\n处理后训练集大小: {len(train_data)}")
print(f"处理后测试集大小: {len(test_data)}")

# 数据可视化分析
# 合并训练集和测试集用于可视化
visualization_data = pd.concat([train_data, test_data])

plt.figure(figsize=(12, 8))
sns.boxplot(x='Region', y='MSW_log', data=visualization_data)
plt.title('不同地区的MSW分布(对数尺度)')
plt.xticks(rotation=45)
plt.tight_layout()

plt.close()

plt.figure(figsize=(12, 8))
sns.boxplot(x='Income Group', y='MSW_log', data=visualization_data)
plt.title('不同收入组的MSW分布(对数尺度)')
plt.xticks(rotation=45)
plt.tight_layout()

plt.close()

In [ ]:
# 创建时间外样本测试集
# 从训练集中每个国家抽取最后15%的年份数据

# 获取训练集中的国家列表
train_countries = train_data['Country Name'].unique()
print(f"训练集国家数量: {len(train_countries)}")

# 初始化时间外样本测试集
time_test_data = pd.DataFrame()

# 对每个国家，提取最后15%的年份数据
for country in train_countries:
    country_data = train_data[train_data['Country Name'] == country].copy()
    
    # 按年份排序
    country_data = country_data.sort_values('Year')
    
    # 计算要提取的数据量
    n_years = len(country_data)
    n_test = max(1, int(n_years * 0.15))  # 至少提取1个样本
    
    # 提取最后15%的数据
    country_test = country_data.iloc[-n_test:].copy()
    time_test_data = pd.concat([time_test_data, country_test])

# 从训练集中移除这些数据，创建新的训练集
final_train_data = train_data[~train_data.index.isin(time_test_data.index)].copy()

# 重命名变量，使命名更加清晰
country_test_data = test_data  # 原来的测试集，按国家划分的

print(f"时间外样本测试集大小: {len(time_test_data)}")
print(f"国家外样本测试集大小: {len(country_test_data)}")
print(f"最终训练集大小: {len(final_train_data)}")
print(f"总训练集大小: {len(train_data)}")

# 查看时间外样本测试集的年份分布
time_test_years = time_test_data['Year'].value_counts().sort_index()
print("\n时间外样本测试集年份分布:")
print(time_test_years)

In [ ]:
country_test_data

In [ ]:
# 准备PyCaret所需的数据格式
columns_to_drop = ['MSW', 'Country Name', 'Year']
final_train_pycaret = final_train_data.drop(columns=columns_to_drop)
time_test_pycaret = time_test_data.drop(columns=columns_to_drop)
country_test_pycaret = country_test_data.drop(columns=columns_to_drop)

# 使用PyCaret进行模型训练
from pycaret.regression import *

# # 如果您想兼顾两种能力，可以考虑混合测试集
# mixed_test = pd.concat([time_test_pycaret.sample(frac=0.5), 
#                         country_test_pycaret.sample(frac=0.5)]).reset_index(drop=True)

# 设置PyCaret环境 - 保持对数变换处理
setup(data=final_train_pycaret, 
      target='MSW_log',  # 继续使用对数变换后的目标变量
      test_data=country_test_pycaret,
      session_id=456,
      fold_strategy='timeseries',
      fold=5,
      normalize=True,
      normalize_method='minmax',  # 使用minmax归一化
      data_split_shuffle=False,
      fold_shuffle=False,
      categorical_features=['Region', 'Income Group'],
      log_experiment=True,
      experiment_name='msw_prediction')

# 比较不同模型并选择前4个最佳模型
top_models = compare_models(n_select=4, sort='mape')
print("选择的顶级模型:")
for i, model in enumerate(top_models):
    print(f"模型 {i+1}: {model}")



,Description,Value
0,Session id,456
1,Target,MSW_log
2,Target type,Regression
3,Original data shape,"(1755, 19)"
4,Transformed data shape,"(1755, 28)"
5,Transformed train set shape,"(1483, 28)"
6,Transformed test set shape,"(272, 28)"
7,Numeric features,16
8,Categorical features,2
9,Preprocess,True


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,16:42:20
Status,. . . . . . . . . . . . . . . . . .,Initializing CV
Estimator,. . . . . . . . . . . . . . . . . .,Linear Regression


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)


Processing:   0%|          | 0/88 [00:00<?, ?it/s]

In [ ]:
# 对每个模型进行调优
tuned_models = []
for i, model in enumerate(top_models):
    print(f"\n调优模型 {i+1}...")
    tuned_model = tune_model(model, n_iter=100, optimize = 'R2',search_library = 'optuna',
                            early_stopping = 20)
    predict_model(tuned_model)
    tuned_models.append(tuned_model)
    # 保存调优后的模型名称变量
    exec(f"tunedmodeltop{i+1} = tuned_model")

In [ ]:

# # 使用MLPRegressor作为元模型进行堆叠
# from sklearn.neural_network import MLPRegressor

# # 定义MLP元模型
# mlp_meta_model = MLPRegressor(
#     hidden_layer_sizes=(100, 50),  # 两层隐藏层
#     max_iter=2000,
#     solver='adam',
#     alpha=0.005,
#     learning_rate='adaptive',
#     learning_rate_init=0.001,
#     tol=1e-5,
#     random_state=456
# )

selected_models = [tuned_models[0],tuned_models[2]]

# 使用堆叠模型集成
# print("\n创建堆叠模型...")
# stacked_model = stack_models(estimator_list=selected_models)

print("\n创建投票集成模型...")
weights = [0.1, 0.4, 0.4, 0.1]  # 根据模型性能分配权重
from sklearn.ensemble import VotingRegressor  # 导入VotingRegressor
estimators = [(f'model_{i}', model) for i, model in enumerate(selected_models)]  # 将模型列表转换为元组列表格式
# 使用VotingRegressor创建集成模型
# 创建投票回归器模型
ensemble_model = VotingRegressor(estimators=estimators)
# 将ensemble_model赋值给stacked_model，不调用它作为函数
# 最终模型评估
final_model = finalize_model(ensemble_model)

# 在时间外样本测试集上评估模型
time_test_predictions = predict_model(final_model, data=time_test_pycaret)
time_test_r2 = r2_score(time_test_pycaret['MSW_log'], time_test_predictions['prediction_label'])
time_test_mse = mean_squared_error(time_test_pycaret['MSW_log'], time_test_predictions['prediction_label'])

# 在国家外样本测试集上评估模型
country_test_predictions = predict_model(final_model, data=country_test_pycaret)
country_test_r2 = r2_score(country_test_pycaret['MSW_log'], country_test_predictions['prediction_label'])
country_test_mse = mean_squared_error(country_test_pycaret['MSW_log'], country_test_predictions['prediction_label'])

# 评估对数空间和原始空间的性能
# 将预测结果转换回原始空间
time_test_results = time_test_data.copy()
time_test_results['MSW_pred'] = np.expm1(time_test_predictions['prediction_label'])
country_test_results = country_test_data.copy()
country_test_results['MSW_pred'] = np.expm1(country_test_predictions['prediction_label'])

# 计算原始空间的性能指标
time_original_r2 = r2_score(time_test_data['MSW'], time_test_results['MSW_pred'])
time_original_mse = mean_squared_error(time_test_data['MSW'], time_test_results['MSW_pred'])
country_original_r2 = r2_score(country_test_data['MSW'], country_test_results['MSW_pred'])
country_original_mse = mean_squared_error(country_test_data['MSW'], country_test_results['MSW_pred'])

# 计算平均绝对百分比误差(MAPE)
time_test_results['Error_percent'] = np.abs((time_test_results['MSW_pred'] - time_test_results['MSW']) / time_test_results['MSW']) * 100
country_test_results['Error_percent'] = np.abs((country_test_results['MSW_pred'] - country_test_results['MSW']) / country_test_results['MSW']) * 100

time_mape = time_test_results['Error_percent'].mean()
country_mape = country_test_results['Error_percent'].mean()

# 输出评估结果
print("\n===== 模型评估结果 =====")
print("\n时间外样本测试集评估:")
print(f"对数空间 - MSE: {time_test_mse:.4f}, R²: {time_test_r2:.4f}")
print(f"原始空间 - MSE: {time_original_mse:.2f}, R²: {time_original_r2:.4f}")
print(f"MAPE: {time_mape:.2f}%")

print("\n国家外样本测试集评估:")
print(f"对数空间 - MSE: {country_test_mse:.4f}, R²: {country_test_r2:.4f}")
print(f"原始空间 - MSE: {country_original_mse:.2f}, R²: {country_original_r2:.4f}")
print(f"MAPE: {country_mape:.2f}%")

# 保存最终模型
save_model(final_model, 'msw_stacked_model')
print("\n最终堆叠模型已保存为 'msw_stacked_model'")

In [ ]:
# 时间外样本测试集 - 各国家历史数据与预测数据对比图

# 将预测结果与原始数据合并
time_test_results = time_test_data.copy()
time_test_results['MSW_pred'] = np.expm1(time_test_predictions['prediction_label'])

# 选择几个代表性国家进行可视化
sample_countries = time_test_results['Country Name'].value_counts().head(6).index.tolist()

# 获取这些国家的完整历史数据（包括训练集和测试集）
full_data = pd.concat([final_train_data, time_test_data])

# 为每个样本国家绘制时间序列图
plt.figure(figsize=(18, 15))

for i, country in enumerate(sample_countries):
    plt.subplot(3, 2, i+1)
    
    # 获取该国家的数据
    country_full = full_data[full_data['Country Name'] == country].sort_values('Year')
    country_train = final_train_data[final_train_data['Country Name'] == country].sort_values('Year')
    country_test = time_test_results[time_test_results['Country Name'] == country].sort_values('Year')
    
    # 绘制历史数据
    plt.plot(country_train['Year'], country_train['MSW'], 'o-', color='blue', label='历史数据')
    
    # 绘制测试集实际值
    plt.plot(country_test['Year'], country_test['MSW'], 'o-', color='green', label='实际值')
    
    # 绘制测试集预测值
    plt.plot(country_test['Year'], country_test['MSW_pred'], 'x--', color='red', label='预测值')
    
    plt.title(f'{country}的MSW时间序列')
    plt.xlabel('年份')
    plt.ylabel('MSW')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

# 绘制所有国家的预测误差分布
time_test_results['Error_percent'] = ((time_test_results['MSW_pred'] - time_test_results['MSW']) / time_test_results['MSW']) * 100

plt.figure(figsize=(10, 6))
plt.hist(time_test_results['Error_percent'], bins=20, alpha=0.7)
plt.axvline(x=0, color='r', linestyle='--')
plt.xlabel('预测误差百分比 (%)')
plt.ylabel('频数')
plt.title('时间外样本测试集 - 预测误差百分比分布')
plt.grid(True, linestyle='--', alpha=0.7)
plt.show()

# 绘制预测误差与年份的关系
plt.figure(figsize=(12, 6))
plt.scatter(time_test_results['Year'], time_test_results['Error_percent'], alpha=0.6)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('年份')
plt.ylabel('预测误差百分比 (%)')
plt.title('时间外样本测试集 - 预测误差与年份的关系')
plt.grid(True, linestyle='--', alpha=0.7)
plt.show()

In [ ]:
# 国家外样本测试集 - 各国家随时间的趋势图

# 将预测结果与原始数据合并
country_test_results = country_test_data.copy()
country_test_results['MSW_pred'] = np.expm1(country_test_predictions['prediction_label'])

# 选择几个代表性国家进行可视化
sample_countries = country_test_results['Country Name'].value_counts().head(6).index.tolist()

# 为每个样本国家绘制时间序列图
plt.figure(figsize=(18, 15))

for i, country in enumerate(sample_countries):
    plt.subplot(3, 2, i+1)
    
    # 获取该国家的数据
    country_data = country_test_results[country_test_results['Country Name'] == country].sort_values('Year')
    
    # 绘制实际值
    plt.plot(country_data['Year'], country_data['MSW'], 'o-', color='blue', label='实际值')
    
    # 绘制预测值
    plt.plot(country_data['Year'], country_data['MSW_pred'], 'x--', color='red', label='预测值')
    
    plt.title(f'{country}的MSW时间序列')
    plt.xlabel('年份')
    plt.ylabel('MSW (kg/人/年)')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

# 绘制所有国家的预测误差随时间变化
plt.figure(figsize=(14, 8))

# 计算每个国家每年的预测误差百分比
country_test_results['Error_percent'] = ((country_test_results['MSW_pred'] - country_test_results['MSW']) / country_test_results['MSW']) * 100

# 按年份分组计算平均误差
yearly_error = country_test_results.groupby('Year')['Error_percent'].agg(['mean', 'std']).reset_index()

# 绘制平均误差随时间变化的趋势
plt.errorbar(yearly_error['Year'], yearly_error['mean'], yerr=yearly_error['std'], 
             fmt='o-', capsize=5, ecolor='gray', alpha=0.7)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('年份')
plt.ylabel('平均预测误差百分比 (%)')
plt.title('国家外样本测试集 - 预测误差随时间变化')
plt.grid(True, linestyle='--', alpha=0.7)
plt.show()

# 绘制所有国家的实际值与预测值散点图
plt.figure(figsize=(10, 8))
plt.scatter(country_test_results['MSW'], country_test_results['MSW_pred'], alpha=0.6)
plt.plot([country_test_results['MSW'].min(), country_test_results['MSW'].max()], 
         [country_test_results['MSW'].min(), country_test_results['MSW'].max()], 
         'r--')
plt.xlabel('实际MSW值')
plt.ylabel('预测MSW值')
plt.title('国家外样本测试集 - 实际值与预测值对比')
plt.grid(True, linestyle='--', alpha=0.7)
plt.show()

# 按地区分组绘制预测误差箱线图
plt.figure(figsize=(14, 8))
sns.boxplot(x='Region', y='Error_percent', data=country_test_results)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('地区')
plt.ylabel('预测误差百分比 (%)')
plt.title('国家外样本测试集 - 各地区预测误差分布')
plt.xticks(rotation=45, ha='right')
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# 设置中文字体支持
plt.rcParams['font.sans-serif'] = ['SimHei', 'Microsoft YaHei', 'SimSun', 'Arial Unicode MS']  # 尝试多种中文字体
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号

def plot_country_trends(country_name, data=country_test_data, predictions=country_test_results):
    """
    绘制指定国家的MSW、人口和GDP随时间的变化趋势图，以及MSW预测值
    
    参数:
    country_name: 国家名称
    data: 包含国家数据的DataFrame
    predictions: 包含预测结果的DataFrame
    """
    # 筛选指定国家的数据
    country_data = data[data['Country Name'] == country_name].sort_values('Year')
    country_pred = predictions[predictions['Country Name'] == country_name].sort_values('Year')
    
    if len(country_data) == 0:
        print(f"未找到国家 '{country_name}' 的数据")
        return
    
    # 创建一个包含5个子图的图表
    fig, axes = plt.subplots(3, 2, figsize=(18, 15))
    fig.suptitle(f"{country_name}的趋势分析", fontsize=16)
    
    # 1. MSW实际值和预测值趋势图
    axes[0, 0].plot(country_data['Year'], country_data['MSW'], 'o-', color='blue', label='实际值')
    axes[0, 0].plot(country_pred['Year'], country_pred['MSW_pred'], 'x--', color='red', label='预测值')
    axes[0, 0].set_title('MSW实际值与预测值对比')
    axes[0, 0].set_xlabel('年份')
    axes[0, 0].set_ylabel('MSW')
    axes[0, 0].legend()
    axes[0, 0].grid(True, linestyle='--', alpha=0.7)
    # 2. 人口趋势图
    axes[0, 1].plot(country_data['Year'], country_data['Population'], 'o-', color='green')
    axes[0, 1].set_title('人口随时间变化')
    axes[0, 1].set_xlabel('年份')
    axes[0, 1].set_ylabel('人口')
    axes[0, 1].grid(True, linestyle='--', alpha=0.7)
    
    # 3. GDP PPP趋势图
    axes[1, 0].plot(country_data['Year'], country_data['GDP PPP 2017'], 'o-', color='red')
    axes[1, 0].set_title('GDP PPP随时间变化')
    axes[1, 0].set_xlabel('年份')
    axes[1, 0].set_ylabel('GDP PPP (2017)')
    axes[1, 0].grid(True, linestyle='--', alpha=0.7)
    
    # 4. GDP PPP/capita趋势图
    axes[1, 1].plot(country_data['Year'], country_data['GDP PPP/capita 2017'], 'o-', color='purple')
    axes[1, 1].set_title('人均GDP PPP随时间变化')
    axes[1, 1].set_xlabel('年份')
    axes[1, 1].set_ylabel('GDP PPP/capita (2017)')
    axes[1, 1].grid(True, linestyle='--', alpha=0.7)
    
    # 5. MSW预测误差百分比
    if 'Error_percent' in country_pred.columns:
        axes[2, 0].bar(country_pred['Year'], country_pred['Error_percent'], color='orange')
        axes[2, 0].set_title('MSW预测误差百分比')
        axes[2, 0].set_xlabel('年份')
        axes[2, 0].set_ylabel('误差百分比 (%)')
        axes[2, 0].grid(True, linestyle='--', alpha=0.7)
        axes[2, 0].axhline(y=0, color='r', linestyle='--')
    
    # 6. MSW与GDP PPP/capita的关系
    axes[2, 1].scatter(country_data['GDP PPP/capita 2017'], country_data['MSW'], color='blue')
    axes[2, 1].set_title('MSW与人均GDP PPP的关系')
    axes[2, 1].set_xlabel('GDP PPP/capita (2017)')
    axes[2, 1].set_ylabel('MSW')
    axes[2, 1].grid(True, linestyle='--', alpha=0.7)
    
    plt.tight_layout(rect=[0, 0, 1, 0.96])  # 调整布局，为总标题留出空间
    plt.show()

# 获取国家测试集中的所有国家列表
test_countries = country_test_data['Country Name'].unique()
print("国家测试集中的国家列表:")
for i, country in enumerate(test_countries):
    print(f"{i+1}. {country}")

# 使用示例
# 绘制第一个国家的趋势图
if len(test_countries) > 0:
    plot_country_trends(test_countries[9])

# 您可以通过以下方式查看任意国家的趋势
# plot_country_trends('Malaysia')
    # ... 其余代码保持不变 ...